In [27]:
import pandas
import numpy as np
import json
import glob
import seaborn
import gab

import re
import collections
import urllib
import datetime
import dateutil

import pandas
import matplotlib.pyplot as plt
import numpy as np
import seaborn

import ipywidgets as widgets
from IPython.display import display

import json
import gensim

from gensim.models import LdaSeqModel
from gensim.models.wrappers import DtmModel

shootingDate = pandas.Timestamp('2018-10-27')

posts = glob.glob('../2019-03-17_processed_2/posts*')
users = glob.glob('../2019-03-17_processed_2/nodes*')

In [2]:
def loadFile(filePath):
    #print(f"Reading: {filePath}")
    targets = []
    with open(filePath) as f:
        for i, l in enumerate(f):
            targets.append(json.loads(l))
            if i % 10000 == 0:
                print(f"{filePath}:{i}", end = '\r')
    return pandas.DataFrame(targets)

def loadngrams(path):
    with open(path) as f:
        dat = {}
        f.readline()
        for l in f:
            *w, c = l.strip().split(',')
            dat[tuple(w)] = int(c)
    return dat

In [3]:
print("Loading data")
with multiprocessing.Pool(48) as pool:
    print("Loading posts")
    postsDat = pool.map(loadFile, posts)
    print("Loading users")
    usersDat = pool.map(loadFile, users)

Loading data
Loading posts
Loading users


In [7]:
df_users = pandas.concat(usersDat, sort=False)
df_posts = pandas.concat(postsDat, sort=False)

In [8]:
df_posts['tokens'] = df_posts['clean_body'].apply(lambda x : x.lower().split())

In [9]:
unigrams = {k[0]: v for k, v in loadngrams("../2019-03-17_ngrams_2/1-grams.csv").items()}

In [10]:
df_posts['tokens_reduced'] = df_posts['tokens'].apply(lambda x : [w for w in x if w in unigrams])

In [11]:
df_posts['date'] = df_posts['created_at'].apply(lambda x : x[:7])

In [12]:
df_posts = df_posts.sort_values('date')

In [14]:
targets = []
def getDate(target):
    df_working = df_posts[df_posts['date'] == target]
    return df_working.iloc[:100].copy()
with multiprocessing.Pool(24) as pool:
    reduced = pool.map(getDate, df_posts['date'].unique())

In [15]:
targets = []
for d in reduced:
    targets += list(d['tokens_reduced'])

In [30]:
df_counts = df_posts[['date','body']].groupby('date').count()

In [16]:
dictionary = gensim.corpora.Dictionary(targets)

In [17]:
with multiprocessing.Pool(28) as pool:
    corpus = pool.map(dictionary.doc2bow, targets)

In [19]:
dictionary.save('dlda_dict.mm')

In [23]:
corpus[-1]

[(270, 1)]

In [24]:
with open('dlda_corpus.json', 'w') as f:
    json.dump(corpus,f)

In [37]:
time_slices = list(df_counts['body'])
time_slices = [len(t) for t in reduced]

In [38]:
with open('dlda_time_slices.json', 'w') as f:
    json.dump(time_slices,f)

In [ ]:
model = DtmModel(
    '/u/reidmcy/.local/bin/dtm-linux64', corpus=corpus, id2word=dictionary,
    time_slices=time_slices,
)

In [ ]:
model.save('DtmModel.mm')

In [ ]:
ldaseq = LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slices, num_topics=100)

In [1]:
ldaseq

NameError: name 'ldaseq' is not defined